In [1]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt

from collections import OrderedDict

In [311]:
###################################
# A Class to store battery functionaly such as revenue, charge and discharge
# periods, charge and discharge spot prices, charge and discharge market dispatch
#
# NOTE : import numpy as np

OPENING = 0
CLOSING = 1

class Battery:
    """
    A Class to store battery functionaly such as revenue, charge and discharge
    periods, charge and discharge spot prices, charge and discharge market dispatch.

    ```NOTE``` : IF THERE IS ANY CHANGE IN CALCULATION, CHANGE CALCULATION IN THE FUNCTION
     MARKED BY (BATTERY CALCULATION).

    ---
    Consists of:
      - charge_period : period when it should charge.
      - discharge_period : period when it should discharge.
      - charge_price : spot price given charging period.
      - discharge_price : spot price given discharging period.
      - charge_market_dispatch : set amount of market dispatch given charging period.
      - discharge_market_dispatch : set amount of market dispatch given discharging period.

    ---
    Functions:
      - ComputeRevenue: To calculate revenue given discharge and charge period pairs.
      - Setting : Set all the battery functionalities. (BATTERY CALCULATION)
      - FirstOptimisation : Ensure that energy are not wasted or not used. (BATTERY CALCULATION)
      - SecondOptimisation : Ensure that to always charge the highest price less and discharge
                             highest price more. (BATTERY CALCULATION)
    ---
    Created and Developed by: Gilbert Putra
    """
    # BATTERY SPECIFICATIONS
    mlf = 0.991                 # Marginal Loss Factor
    battery_capacity = 580      # Battery Capacity
    battery_power = 300         # Battery Power
    charge_efficiency = 0.9     # Charge Efficiency
    discharge_efficiency = 0.9  # Discharge Efficiency
        
    def __init__(self, charge_period, charge_spot_price,
                 discharge_period, discharge_spot_price):
        """
        Initialise Battery Class.

        Parameters
        ----------

        charge_period : list (int)
            Possible ranked charging period. Ranked from lowest price to highest.
        charge_spot_price : list (int)
            Spot price of the possible charge period.
        discharge_period : list (int)
            Possible ranked discharge period. Ranked from highest price to lowest.
        discharge_spot_price : list (int)
            Spot price of the possible discharge period.
        """ 
        # Charge and Discharge Period
        self.charge_period = charge_period
        self.discharge_period = discharge_period
        
        # Spot Price during Charge and Discharge
        self.charge_price = charge_spot_price
        self.discharge_price = discharge_spot_price
    
    ########################################################################################    
    # CALCULATE REVENUE OF THE RESPECTIVE BATTERY.            
    def Revenue(self):
        if self.charge_market_dispatch == [] or self.discharge_market_dispatch == []:
            return 0
        
        # Spot Prices
        charge_sp = np.array(self.charge_price)[:, 1]
        discharge_sp = np.array(self.discharge_price)[:, 1]

        # Market Dispatches
        charge_md = np.array(self.charge_market_dispatch).T
        discharge_md = np.array(self.discharge_market_dispatch).T

        # Revenues
        charge_revenue = (charge_sp @ charge_md) * (1 / self.mlf)
        discharge_revenue = (discharge_sp @ discharge_md) * (self.mlf)

        return discharge_revenue + charge_revenue        

    def Setting(self, discharge_capacity, charge_capacity):
        if len(self.charge_period) == len(self.discharge_period):
            self.EqualSetting(discharge_capacity, charge_capacity)
        else:
            self.NonEqualSetting(discharge_capacity, charge_capacity)
            
    def EqualSetting(self, discharge_capacity, charge_capacity):
        battery_power = self.battery_power
        battery_cap = np.amax(charge_capacity)
        
        len_charge = len(self.charge_period)
        len_discharge = len(self.discharge_period)
        
        MAX_CHARGE_PERIOD = 5
        MAX_DISCHARGE_PERIOD = 4
        
        MAX_RAW_POWER = 300
        MAX_CHARGE_MD = MAX_RAW_POWER / 2
        MAX_DISCHARGE_MD = MAX_RAW_POWER / 2 * self.discharge_efficiency
        
        self.charge_capacity = charge_capacity
        self.discharge_capacity = discharge_capacity
        
        self.charge_raw_power = ['' for i in range(len_charge)]
        self.discharge_raw_power = ['' for i in range(len_discharge)]
        
        self.charge_market_dispatch = ['' for i in range(len_charge)]
        self.discharge_market_dispatch = ['' for i in range(len_discharge)]
        
        self.charge_capacity[-1][OPENING] = max(0, min(self.charge_capacity[-1][CLOSING] - MAX_CHARGE_MD * self.charge_efficiency, battery_cap))
                                                
        for t in range(1, len_charge):
            self.charge_capacity[-t -1][CLOSING] = self.charge_capacity[-t][OPENING]
            self.charge_capacity[-t -1][OPENING] = max(0, min(self.charge_capacity[-t -1][CLOSING] - MAX_CHARGE_MD * self.charge_efficiency, battery_cap))
            
            self.charge_market_dispatch[-t] = -min((self.charge_capacity[-t][CLOSING] - 
                                                    self.charge_capacity[-t][OPENING]) / self.charge_efficiency, MAX_CHARGE_MD)
            self.charge_raw_power[-t] = max(-MAX_RAW_POWER, self.charge_market_dispatch[-t] * 2)
            
        self.charge_market_dispatch[0] = -min((self.charge_capacity[0][CLOSING] - 
                                               self.charge_capacity[0][OPENING]) / self.charge_efficiency, MAX_CHARGE_MD)
        self.charge_raw_power[0] = max(-MAX_RAW_POWER, self.charge_market_dispatch[0] * 2)  
        
        self.discharge_capacity[-1][CLOSING] = self.charge_capacity[0][OPENING]
        self.discharge_capacity[-1][OPENING] = min(battery_cap, self.discharge_capacity[-1][CLOSING] + MAX_DISCHARGE_MD / self.discharge_efficiency)
        self.discharge_market_dispatch[-1] = min(MAX_DISCHARGE_MD, (self.discharge_capacity[-1][OPENING] - self.discharge_capacity[-1][CLOSING]) * self.discharge_efficiency) 
        self.discharge_raw_power[-1] = min(MAX_RAW_POWER, self.discharge_market_dispatch[-1] * 2 / self.discharge_efficiency)
        for t in range(1, len_discharge):
            self.discharge_capacity[-t -1][CLOSING] = self.discharge_capacity[-t][OPENING]
            self.discharge_capacity[-t -1][OPENING] = min(battery_cap, self.discharge_capacity[-t -1][CLOSING] + MAX_DISCHARGE_MD / self.discharge_efficiency)
            self.discharge_market_dispatch[-t-1] = min(MAX_DISCHARGE_MD, (self.discharge_capacity[-t -1][OPENING] - self.discharge_capacity[-t -1][CLOSING]) * self.discharge_efficiency) 
            self.discharge_raw_power[-t -1] = min(MAX_RAW_POWER, self.discharge_market_dispatch[-t -1] * 2 / self.discharge_efficiency)    

        #print("EQUAL")
        #print("DISCHARGE")
        #print(self.discharge_raw_power)
        #print(self.discharge_market_dispatch)
        #print(self.discharge_capacity)
        #print("CHARGE")
        #print(self.charge_raw_power)
        #print(self.charge_market_dispatch)
        #print(self.charge_capacity)
        #print("_________")
        
    def NonEqualSetting(self, discharge_capacity, charge_capacity):
        battery_power = self.battery_power
        battery_cap = np.amax(charge_capacity)
        
        len_charge = len(self.charge_period)
        len_discharge = len(self.discharge_period)
        
        MAX_CHARGE_PERIOD = 5
        MAX_DISCHARGE_PERIOD = 4
        
        MAX_RAW_POWER = 300
        MAX_CHARGE_MD = MAX_RAW_POWER / 2
        MAX_DISCHARGE_MD = MAX_RAW_POWER / 2 * self.discharge_efficiency
        
        self.charge_capacity = charge_capacity
        self.discharge_capacity = discharge_capacity
        
        self.charge_raw_power = ['' for i in range(len_charge)]
        self.discharge_raw_power = ['' for i in range(len_discharge)]
        
        self.charge_market_dispatch = ['' for i in range(len_charge)]
        self.discharge_market_dispatch = ['' for i in range(len_discharge)]
                
        self.discharge_capacity[0][CLOSING] = max(0, min(self.discharge_capacity[0][OPENING] - 
                                                         MAX_DISCHARGE_MD / self.discharge_efficiency, battery_cap))
        #self.discharge_market_dispatch[0] = min((self.discharge_capacity[0][OPENING] - 
        #                                        self.discharge_capacity[0][CLOSING]) * self.discharge_efficiency, MAX_DISCHARGE_MD)
        #self.discharge_raw_power[0] = min(MAX_RAW_POWER, self.discharge_market_dispatch[0] * 2 / 0.9)
        
        for t in range(1, len_discharge):
            self.discharge_capacity[t][OPENING] = self.discharge_capacity[t - 1][CLOSING]
            self.discharge_capacity[t][CLOSING] = max(0, min(self.discharge_capacity[t][OPENING] - MAX_DISCHARGE_MD / self.discharge_efficiency, battery_cap))
            self.discharge_market_dispatch[t-1] = min((self.discharge_capacity[t-1][OPENING] - 
                                                    self.discharge_capacity[t-1][CLOSING]) * self.discharge_efficiency, MAX_DISCHARGE_MD)
            self.discharge_raw_power[t-1] = min(MAX_RAW_POWER, self.discharge_market_dispatch[t-1] * 2 / 0.9)
        
        self.discharge_market_dispatch[-1] = min((self.discharge_capacity[-1][OPENING] - 
                                                self.discharge_capacity[-1][CLOSING]) * self.discharge_efficiency, MAX_DISCHARGE_MD)
        self.discharge_raw_power[-1] = min(MAX_RAW_POWER, self.discharge_market_dispatch[-1] * 2 / 0.9)
        
        self.charge_capacity[0][OPENING] = self.discharge_capacity[-1][CLOSING]
        self.charge_market_dispatch[0] = min(self.charge_capacity[0][CLOSING] - 
                                             self.charge_capacity[0][OPENING] * self.charge_efficiency, MAX_CHARGE_MD)
        
        self.charge_raw_power[0] = min(MAX_RAW_POWER, self.charge_market_dispatch[0] * 2)
        for t in range(1, len_charge):
            self.charge_capacity[t - 1][CLOSING] = min(battery_cap, self.charge_capacity[t-1][OPENING] + MAX_CHARGE_MD * self.charge_efficiency)
            self.charge_capacity[t][OPENING] = self.charge_capacity[t - 1][CLOSING]
            
            self.charge_market_dispatch[t-1] = -min((self.charge_capacity[t-1][CLOSING] - 
                                                 self.charge_capacity[t-1][OPENING]) / self.charge_efficiency, MAX_CHARGE_MD)
            self.charge_raw_power[t-1] = max(-MAX_RAW_POWER, self.charge_market_dispatch[t-1] * 2)
            
        self.charge_market_dispatch[-1] = -min((self.charge_capacity[-1][CLOSING] - 
                                                 self.charge_capacity[-1][OPENING]) / self.charge_efficiency, MAX_CHARGE_MD)
        self.charge_raw_power[-1] = max(-MAX_RAW_POWER, self.charge_market_dispatch[-1] * 2)
    
    def SecondOptimisation(self):
        OPENING = 0
        CLOSING = 1
        
        battery_power = self.battery_power
        battery_cap = self.battery_capacity
        
        len_charge = len(self.charge_period)
        len_discharge = len(self.discharge_period)
        
        MAX_CHARGE_PERIOD = 5
        MAX_DISCHARGE_PERIOD = 4
        
        # CHARGE PERIOD --------------------------------------------------------------------
        highest_price_index = np.array(self.charge_period).argmax(axis=0)[0]
        lowest_charge_index = self.charge_market_dispatch.index(max(self.charge_market_dispatch))    
        
        # IF THE HIGHEST CHARGING PRICE DOESN'T HAVE THE LOWEST CHARGING RATE, SWAP!
        if (highest_price_index != lowest_charge_index):
            tmp = self.charge_market_dispatch[highest_price_index]
            self.charge_market_dispatch[highest_price_index] = self.charge_market_dispatch[lowest_charge_index]
            self.charge_market_dispatch[lowest_charge_index] = tmp
            # SET THE UPDATED BATTERY SETTINGS.
            for t in range(len_charge):
                # RAW_POWER[t] = MARKET_DISPATCH[t] * 2
                self.charge_raw_power[t] = self.charge_market_dispatch[t] * 2

                # CLOSING_CAPACITY[t] = MAX(0, MIN(OPENING_CAPACITY[t] - 
                #                        MARKET_DISPATCH[t] * CHARGE_EFFICIENCY, BATTERY_CAPACITY))
                self.charge_capacity[t][CLOSING] = max(0, min(self.charge_capacity[t][OPENING] - 
                                                            self.charge_market_dispatch[t] * self.charge_efficiency, 
                                                            battery_cap))
                # Ensuring that it doesn't exceeds array len limit
                if t + 1 < len_charge:
                    self.charge_capacity[t + 1][OPENING] = self.charge_capacity[t][CLOSING]

        # DISCHARGE PERIOD -----------------------------------------------------------------
        lowest_price_index = np.array(self.discharge_period).argmax(axis=0)[0] 
        min_discharge_index = self.discharge_market_dispatch.index(min(self.discharge_market_dispatch))   
        
        # IF THE LOWEST DISCHARGING PRICE DOESN'T HAVE THE LOWEST DISCHARGING RATE, SWAP!
        if (lowest_price_index != min_discharge_index):
            tmp = self.discharge_market_dispatch[lowest_price_index]
            self.discharge_market_dispatch[lowest_price_index] = self.discharge_market_dispatch[min_discharge_index]
            self.discharge_market_dispatch[min_discharge_index] = tmp
            # SET THE UPDATED BATTERY SETTINGS.
            for t in range(len_discharge):
                # RAW_POWER[t] = MARKET_DISPATCH[t] * 2 / DISCHARGE_EFFICIENCY
                self.discharge_raw_power[t] = self.discharge_market_dispatch[t] * 2 / self.discharge_efficiency

                # CLOSING_CAPACITY[t] = MAX(0, MIN(OPENING_CAPACITY[t] - 
                #                        MARKET_DISPATCH[t] * CHARGE_EFFICIENCY, BATTERY_CAPACITY))
                self.discharge_capacity[t][CLOSING] = max(0, min(self.discharge_capacity[t][OPENING] - 
                                                            self.discharge_market_dispatch[t] / self.discharge_efficiency, 
                                                            battery_cap))
                # Ensuring that it doesn't exceeds array len limit
                if t + 1 < len_discharge:
                    self.discharge_capacity[t + 1][OPENING] = self.discharge_capacity[t][CLOSING]
        

In [324]:
EMPTY = ' '
def setStatus(dataframe):
    status = []
    for i in range(len(dataframe)):
        market_d = dataframe.loc[i, 'Market Dispatch (MWh)']
        opening_c = dataframe.loc[i, 'Opening Capacity (MWh)']
        closing_c = dataframe.loc[i, 'Closing Capacity (MWh)']
        
        if market_d > 0:
            status.append('Discharge')
        elif market_d < 0:
            status.append('Charge')
        elif market_d == 0 and (opening_c != 0 or closing_c != 0):
            status.append('Between')
        else:
            status.append('Nothing')
    return status

def selectBetweenPeriod(dataframe):
    betweenIndex = dataframe[dataframe['Status'] == 'Between'].index
    period = []
    tmp = [betweenIndex[0]]
    for i in range(1, len(betweenIndex)):
        prev = betweenIndex[i - 1]
        curr = betweenIndex[i]
        if curr - prev > 1:
            period.append(tmp)
            tmp = [curr]
        else:
            tmp.append(curr)
            
    period.append(tmp)
    
    tmp = []
    for p in range(len(period)):
        if len(period[p]) >= 2:
            tmp.append([period[p][0], period[p][-1]])
    period = tmp
    
    timePeriod = []
    for p in period:
        timePeriod.append((dataframe.iloc[p[0], 0], dataframe.iloc[p[1], 0]))
    
    return timePeriod

###################################
# A function to get spot prices based on selected regions.
#
# Parameters:
#      - data : the targeted dataset, minimum dataset length of 48.
#      - selected_periods : selected period for charging or discharging.
#      - region : the targeted region, default has been set to 'VIC' for mandatory task.
#
# Return:
#      - List of spot prices given period
#
# Created by: Gilbert
###################################
def GetSpotPrice(data, selected_periods = False, region = 'VIC'):
    if region == 'VIC':
        spot_price = data['Regions VIC Trading Price ($/MWh)']
    elif region == 'NSW':
        spot_price = data['Regions NSW Trading Price ($/MWh)']
    elif region == 'SA':
        spot_price = data['Regions SA Trading Price ($/MWh)']
    elif region == 'TAS':
        spot_price = data['Regions TAS Trading Price ($/MWh)']
    
    spot_price = np.array(spot_price)
    
    # Find the spot prices from selected region. Periods are
    # index + 1, therefore to use the index we need to subtract
    # it by 1.
    if (selected_periods):
        retrieved_prices = []
        for period in selected_periods:
            # append(minimum or maximum ranking, spot_price[index])
            retrieved_prices.append((period[1], spot_price[period[1] - 1]))
        return retrieved_prices
    else:
        return spot_price

###################################
# A function to find minimum and maximum point rank given threshold.
#
# Parameters:
#      - data : the targeted dataset, minimum dataset length of 48.
#      - region : the targeted region, default has been set to 'VIC' for mandatory task.
#      - buy_threshold : maximum number of buying point, default has been set to optimise Checkpoint 3.
#      - sell_threshold : maximum number of selling point, default has been set to optimise Checkpoint 3.
#
# Return:
#      - List of selected minimum point, list of selected maximum point
#
# Efficiency: O(3N + NLogN) = O(NLogN)
#
# Created by: Gilbert
###################################
def GetMinMax(data, region = 'VIC', buy_threshold = 5, sell_threshold = 4):
    if region == 'VIC':
        spot_price = data['Regions VIC Trading Price ($/MWh)']
    elif region == 'NSW':
        spot_price = data['Regions NSW Trading Price ($/MWh)']
    elif region == 'SA':
        spot_price = data['Regions SA Trading Price ($/MWh)']
    elif region == 'TAS':
        spot_price = data['Regions TAS Trading Price ($/MWh)']
    
    price = np.array(spot_price)
    minimum_price = np.argsort(price, kind = 'merge*sort') # (O(NlogN)), mergesort the minimum prices.
    maximum_price = minimum_price[::-1][:len(price)] # (O(N)), maximum is the reverse order of minimum.
    
    selected_min_price = [EMPTY for i in minimum_price] # (O(N)), set an empty array for the whole period.
    selected_max_price = [EMPTY for i in minimum_price] # (O(N)), set an empty array for the whole period.
    
    # Select the lowest price spot over the given
    # buy_threshold as the minimum buying point.
    i = 0
    for b_t in range(buy_threshold):
        selected_min_price[minimum_price[i]] = b_t + 1
        i += 1
        
    # Select the highest price spot over the given
    # sell_threshold as the maximum selling point.
    i = 0
    for s_t in range(sell_threshold):
        selected_max_price[maximum_price[i]] = s_t + 1
        i += 1
        
    return selected_min_price, selected_max_price          

def GetCapacity(data, selected_periods = False):
    opening_cap = np.array(data['Opening Capacity (MWh)'])
    closing_cap = np.array(data['Closing Capacity (MWh)'])
    
    if (selected_periods):
        retrieved_cap = []
        for period in selected_periods:
            retrieved_cap.append([opening_cap[period[1] - 1], closing_cap[period[1] - 1]])
        return retrieved_cap
    else:
        return [opening_cap, closing_cap]
    
def findBatteryPairsReverse(buy_period, sell_period, dataframe):
    MAX_SELL_PERIOD = 4 # MAXIMUM SELLING PERIOD PER PAIR
    MAX_BUY_PERIOD = 5 # MAXIMUM BUYING PERIOD PER PAIR
    
    period = len(buy_period)
    
    cap = GetCapacity(dataframe)
    max_cap = max(cap[0])
    #print(max_cap,max_cap//150 + 1)
    battery = []
    sell = OrderedDict() # Initialise battery selling point. (Ordered Dictionary)
    buy = OrderedDict() # Initialise battery buying point. (Orderered Dictionary)
    
    for p in range(period - 1):
        if sell_period[p] != EMPTY and buy_period[p] == EMPTY:
            if len(sell) < MAX_SELL_PERIOD and len(sell) < max_cap // 150 + 1:
                #print("YES")
                sell[sell_period[p]] = sell_period.index(sell_period[p]) + 1
            else:
                max_key = max(sell, key=int)
                if sell_period[p] < max_key:
                    sell.pop(max_key)
                    sell[sell_period[p]] = sell_period.index(sell_period[p]) + 1

        if len(sell) != 0 and sell_period[p] == EMPTY and buy_period[p] != EMPTY:
            if len(buy) < MAX_BUY_PERIOD and len(buy) < math.ceil(len(sell) * 1.25) and len(buy) < max_cap // 135 + 1:
                buy[buy_period[p]] = buy_period.index(buy_period[p]) + 1
            else:
                max_key = max(buy, key=int)
                if buy_period[p] < max_key:
                    buy.pop(max_key)
                    buy[buy_period[p]] = buy_period.index(buy_period[p]) + 1
        if sell_period[p + 1] != EMPTY and len(buy) != 0:
                battery.append([list(sell.items()), list(buy.items())])
                sell = OrderedDict()
                buy = OrderedDict()
    battery.append([list(sell.items()), list(buy.items())]) 

    for b in battery:
        sell_tmp = np.array(b[0])
        buy_tmp = b[1]
        while len(sell_tmp) > len(buy_tmp): 
            row = 0
            index = np.where(sell_tmp[:,0] == sell_tmp[:,0].max())[0][0]
            sell_tmp = np.delete(sell_tmp, index, axis = row)
        b[0] = sell_tmp.tolist() # Change numpy array to list
        if len(b[0]) == 0 or len(b[1]) == 0:
            battery.remove(b)

    return battery

def SetChargeDischarge(data, battery_pairs, region = "VIC"):
    opening_cap = data['Opening Capacity (MWh)']
    closing_cap = data['Closing Capacity (MWh)']
    
    all_batteries = []
    
    for b in battery_pairs:
        sell_period = b[0] 
        buy_period = b[1] 
        if len(sell_period)!= 0 or len(buy_period) != 0:
            sell_price = GetSpotPrice(data, sell_period, region = region)
            buy_price = GetSpotPrice(data, buy_period, region = region)
            battery = Battery(buy_period, buy_price, sell_period, sell_price)

            sell_cap = GetCapacity(data, sell_period)
            buy_cap = GetCapacity(data, buy_period)
            battery.Setting(sell_cap, buy_cap)
            #battery.SecondOptimisation()

            all_batteries.append(battery)

    return all_batteries

###################################
# A function to calculate daily revenue.
#
# Parameters:
#      - all_batteries : List of battery class pairs.
#
# Return:
#      - Daily revenues
#
# Created by: Gilbert
###################################
def ComputeDailyRevenue(all_batteries):
    revenues = 0
    for battery in all_batteries:
        revenues += battery.Revenue()
    return revenues

THRESHOLD = float(1.25709376)    
def localOptimisation(dataframe):
    period = len(dataframe)
    best_batteries = OrderedDict()
    
    for s in range(1, len(dataframe) + 1):
        for b in range(1, len(dataframe) + 1 - s):
            min_price, max_price = GetMinMax(dataframe, buy_threshold = b, sell_threshold = s)
            battery_pairs = findBatteryPairsReverse(min_price, max_price, dataframe)
            all_batteries = SetChargeDischarge(dataframe, battery_pairs)
            dailyrev = ComputeDailyRevenue(all_batteries)
            if dailyrev not in best_batteries:
                best_batteries[dailyrev] = (all_batteries, (b, s))

    # Find the highest revenue amongst possible combinations in that day
    best_revenue = max(best_batteries)
    best_threshold = best_batteries[best_revenue][1]
    #print(best_threshold)
    battery = best_batteries[best_revenue][0] # The Best battery combinations

    # Initialise raw_power, market_dispatch, opening_capacity and closing capacity
    raw_power = [0 for i in range(period)]
    market_dispatch = [0 for i in range(period)]
    opening_capacity = dataframe.loc[:, 'Opening Capacity (MWh)'].to_numpy()
    closing_capacity = dataframe.loc[:, 'Closing Capacity (MWh)'].to_numpy()
    
    # Iterate over battery combinations to set raw_power, market_dispatch,
    # opening_capacity, closing_capacity into an array to be prepared for 
    # merging with the dataset.
    for b in battery:   
        # Charging Period
        for cp in range(len(b.charge_period)):
            raw_power[b.charge_period[cp][1] - 1] = b.charge_raw_power[cp]
            market_dispatch[b.charge_period[cp][1] - 1] = b.charge_market_dispatch[cp]
            opening_capacity[b.charge_period[cp][1] - 1] = b.charge_capacity[cp][0]
            closing_capacity[b.charge_period[cp][1] - 1] = b.charge_capacity[cp][1]
        # Discharge Period
        for dp in range(len(b.discharge_period)):
            raw_power[b.discharge_period[dp][1] - 1] = b.discharge_raw_power[dp]
            market_dispatch[b.discharge_period[dp][1] - 1] = b.discharge_market_dispatch[dp]
            opening_capacity[b.discharge_period[dp][1] - 1] = b.discharge_capacity[dp][0]
            closing_capacity[b.discharge_period[dp][1] - 1] = b.discharge_capacity[dp][1]
            
       
    # Formatting the opening and closing capacity.
    change = 0
    for i in range(1, len(opening_capacity)):
        if closing_capacity[i - 1] != opening_capacity[i]:
            opening_capacity[i] = closing_capacity[i - 1]
            closing_capacity[i] = opening_capacity[i]

    return raw_power, market_dispatch, opening_capacity, closing_capacity, best_revenue

def localMaximisation(dataframe, timePeriod):
    tmp_df = dataframe.copy()
    for period in timePeriod:
        start_t = period[0]
        end_t = period[1]

        data_interval = tmp_df.loc[(tmp_df['Time (UTC+10)'] >= start_t) & 
                                      (tmp_df['Time (UTC+10)'] <= end_t)]
        
        start_index = data_interval.index[0]

        price = GetSpotPrice(data_interval).tolist()
        min_price = (price.index(min(price)), min(price))
        max_price = (price.index(max(price)), max(price))
        
        raw_power, market_dispatch, opening_capacity, closing_capacity, _ = localOptimisation(data_interval)
        print(period)
        for i in range(len(raw_power)):
            tmp_df.loc[i + start_index, 'Raw Power (MW)'] = raw_power[i]
            tmp_df.loc[i + start_index, 'Market Dispatch (MWh)'] = market_dispatch[i]
            tmp_df.loc[i + start_index, 'Opening Capacity (MWh)'] = opening_capacity[i]
            tmp_df.loc[i + start_index, 'Closing Capacity (MWh)'] = closing_capacity[i]

    return tmp_df

In [190]:
data_128 = pd.read_excel("../../preprocessed_data/First Algorithm/example_128_V3.xlsx")
data_128 = data_128[['Time (UTC+10)', 'Regions VIC Trading Price ($/MWh)', 'Raw Power (MW)', 'Market Dispatch (MWh)', 'Opening Capacity (MWh)', 'Closing Capacity (MWh)']]
data_128

,Time (UTC+10),Regions VIC Trading Price ($/MWh),Raw Power (MW),Market Dispatch (MWh),Opening Capacity (MWh),Closing Capacity (MWh)
0,2018-01-01 00:30:00,92.46,0.0,0.0,0.0,0.0
1,2018-01-01 01:00:00,87.62,0.0,0.0,0.0,0.0
2,2018-01-01 01:30:00,73.08,0.0,0.0,0.0,0.0
3,2018-01-01 02:00:00,70.18,0.0,0.0,0.0,0.0
4,2018-01-01 02:30:00,67.43,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
63451,2021-08-14 22:00:00,49.93,-300.0,-150.0,0.0,135.0
63452,2021-08-14 22:30:00,62.86,270.0,121.5,135.0,0.0
63453,2021-08-14 23:00:00,32.26,0.0,0.0,0.0,0.0
63454,2021-08-14 23:30:00,25.10,0.0,0.0,0.0,0.0


In [325]:
data_interval = SM.loc[(SM['Time (UTC+10)'] >= '2018-03-06 00:30:00') & \
                        (SM['Time (UTC+10)'] <= '2018-03-07 00:00:00')].copy()
data_interval = data_interval.reset_index(drop = True)
data_interval
PeriodMaximisation(data_interval)

,Time (UTC+10),Regions VIC Trading Price ($/MWh),Raw Power (MW),Market Dispatch (MWh),Opening Capacity (MWh),Closing Capacity (MWh),Status
0,2018-03-06 00:30:00,58.29,0.000000,0.000000,0.0,0.0,Nothing
1,2018-03-06 01:00:00,56.22,0.000000,0.000000,0.0,0.0,Nothing
2,2018-03-06 01:30:00,53.90,0.000000,0.000000,0.0,0.0,Nothing
3,2018-03-06 02:00:00,46.36,0.000000,0.000000,0.0,0.0,Nothing
4,2018-03-06 02:30:00,17.77,-300.000000,-150.000000,0.0,135.0,Charge
5,2018-03-06 03:00:00,13.28,-300.000000,-150.000000,135.0,270.0,Charge
6,2018-03-06 03:30:00,12.97,-300.000000,-150.000000,270.0,405.0,Charge
7,2018-03-06 04:00:00,12.77,-300.000000,-150.000000,405.0,540.0,Charge
8,2018-03-06 04:30:00,44.78,-88.888889,-44.444444,540.0,580.0,Charge
9,2018-03-06 05:00:00,56.41,0.000000,0.000000,580.0,580.0,Between


In [315]:
status = setStatus(data_interval)
data_interval['Status'] = pd.Series(status)
timePeriod = selectBetweenPeriod(data_interval)
newData = localMaximisation(data_interval, timePeriod)
newData

(1, 1)
(Timestamp('2018-03-06 05:00:00'), Timestamp('2018-03-06 06:00:00'))
(1, 1)
(Timestamp('2018-03-06 07:30:00'), Timestamp('2018-03-06 16:00:00'))


,Time (UTC+10),Regions VIC Trading Price ($/MWh),Raw Power (MW),Market Dispatch (MWh),Opening Capacity (MWh),Closing Capacity (MWh),Status
0,2018-03-06 00:30:00,58.29,0.000000,0.000000,0.0,0.0,Nothing
1,2018-03-06 01:00:00,56.22,0.000000,0.000000,0.0,0.0,Nothing
2,2018-03-06 01:30:00,53.90,0.000000,0.000000,0.0,0.0,Nothing
3,2018-03-06 02:00:00,46.36,0.000000,0.000000,0.0,0.0,Nothing
4,2018-03-06 02:30:00,17.77,-300.000000,-150.000000,0.0,135.0,Charge
5,2018-03-06 03:00:00,13.28,-300.000000,-150.000000,135.0,270.0,Charge
6,2018-03-06 03:30:00,12.97,-300.000000,-150.000000,270.0,405.0,Charge
7,2018-03-06 04:00:00,12.77,-300.000000,-150.000000,405.0,540.0,Charge
8,2018-03-06 04:30:00,44.78,-88.888889,-44.444444,540.0,580.0,Charge
9,2018-03-06 05:00:00,56.41,0.000000,0.000000,580.0,580.0,Between


## FULL DATA

In [326]:
full_status = setStatus(data_128)
data_128['Status'] = pd.Series(full_status)
fullTimePeriod = selectBetweenPeriod(data_128)
new_full = localMaximisation(data_128, fullTimePeriod)
new_full

(Timestamp('2018-01-01 06:30:00'), Timestamp('2018-01-01 15:30:00'))
(Timestamp('2018-01-02 05:00:00'), Timestamp('2018-01-02 08:30:00'))
(Timestamp('2018-01-02 09:30:00'), Timestamp('2018-01-02 11:30:00'))
(Timestamp('2018-01-03 04:00:00'), Timestamp('2018-01-03 11:30:00'))
(Timestamp('2018-01-03 12:30:00'), Timestamp('2018-01-03 13:30:00'))
(Timestamp('2018-01-03 14:30:00'), Timestamp('2018-01-03 18:30:00'))
(Timestamp('2018-01-04 02:30:00'), Timestamp('2018-01-04 03:00:00'))
(Timestamp('2018-01-04 05:00:00'), Timestamp('2018-01-04 07:30:00'))
(Timestamp('2018-01-04 09:30:00'), Timestamp('2018-01-04 10:30:00'))
(Timestamp('2018-01-04 14:00:00'), Timestamp('2018-01-04 15:30:00'))
(Timestamp('2018-01-04 19:30:00'), Timestamp('2018-01-04 20:00:00'))
(Timestamp('2018-01-05 05:30:00'), Timestamp('2018-01-05 06:30:00'))
(Timestamp('2018-01-05 07:30:00'), Timestamp('2018-01-05 16:30:00'))
(Timestamp('2018-01-06 06:00:00'), Timestamp('2018-01-06 15:30:00'))
(Timestamp('2018-01-06 17:30:00'),

(Timestamp('2018-02-10 07:30:00'), Timestamp('2018-02-10 08:00:00'))
(Timestamp('2018-02-10 15:30:00'), Timestamp('2018-02-10 16:30:00'))
(Timestamp('2018-02-11 04:30:00'), Timestamp('2018-02-11 05:30:00'))
(Timestamp('2018-02-11 06:30:00'), Timestamp('2018-02-11 20:30:00'))
(Timestamp('2018-02-11 22:30:00'), Timestamp('2018-02-11 23:00:00'))
(Timestamp('2018-02-12 04:30:00'), Timestamp('2018-02-12 05:30:00'))
(Timestamp('2018-02-12 12:00:00'), Timestamp('2018-02-12 15:00:00'))
(Timestamp('2018-02-12 21:00:00'), Timestamp('2018-02-12 21:30:00'))
(Timestamp('2018-02-12 22:30:00'), Timestamp('2018-02-12 23:30:00'))
(Timestamp('2018-02-13 02:00:00'), Timestamp('2018-02-13 02:30:00'))
(Timestamp('2018-02-13 06:00:00'), Timestamp('2018-02-13 14:30:00'))
(Timestamp('2018-02-13 16:00:00'), Timestamp('2018-02-13 19:30:00'))
(Timestamp('2018-02-14 10:30:00'), Timestamp('2018-02-14 11:00:00'))
(Timestamp('2018-02-14 13:30:00'), Timestamp('2018-02-14 14:00:00'))
(Timestamp('2018-02-14 19:00:00'),

(Timestamp('2018-03-30 14:00:00'), Timestamp('2018-03-30 17:00:00'))
(Timestamp('2018-03-31 01:30:00'), Timestamp('2018-03-31 02:00:00'))
(Timestamp('2018-03-31 03:00:00'), Timestamp('2018-03-31 04:00:00'))
(Timestamp('2018-03-31 05:00:00'), Timestamp('2018-03-31 05:30:00'))
(Timestamp('2018-03-31 13:30:00'), Timestamp('2018-03-31 17:00:00'))
(Timestamp('2018-03-31 19:30:00'), Timestamp('2018-03-31 23:30:00'))
(Timestamp('2018-04-01 04:30:00'), Timestamp('2018-04-01 05:30:00'))
(Timestamp('2018-04-01 07:00:00'), Timestamp('2018-04-01 11:00:00'))
(Timestamp('2018-04-01 12:00:00'), Timestamp('2018-04-01 14:00:00'))
(Timestamp('2018-04-01 15:00:00'), Timestamp('2018-04-01 17:30:00'))
(Timestamp('2018-04-02 06:30:00'), Timestamp('2018-04-02 17:00:00'))
(Timestamp('2018-04-03 06:00:00'), Timestamp('2018-04-03 17:00:00'))
(Timestamp('2018-04-04 05:30:00'), Timestamp('2018-04-04 14:00:00'))
(Timestamp('2018-04-04 22:30:00'), Timestamp('2018-04-04 23:30:00'))
(Timestamp('2018-04-05 02:30:00'),

(Timestamp('2018-05-13 19:00:00'), Timestamp('2018-05-13 23:30:00'))
(Timestamp('2018-05-14 05:00:00'), Timestamp('2018-05-14 06:00:00'))
(Timestamp('2018-05-14 15:30:00'), Timestamp('2018-05-14 17:30:00'))
(Timestamp('2018-05-15 05:30:00'), Timestamp('2018-05-15 06:30:00'))
(Timestamp('2018-05-15 07:30:00'), Timestamp('2018-05-15 08:30:00'))
(Timestamp('2018-05-15 12:30:00'), Timestamp('2018-05-15 14:00:00'))
(Timestamp('2018-05-15 15:00:00'), Timestamp('2018-05-15 16:30:00'))
(Timestamp('2018-05-16 06:00:00'), Timestamp('2018-05-16 06:30:00'))
(Timestamp('2018-05-16 12:00:00'), Timestamp('2018-05-16 12:30:00'))
(Timestamp('2018-05-16 14:00:00'), Timestamp('2018-05-16 14:30:00'))
(Timestamp('2018-05-17 05:30:00'), Timestamp('2018-05-17 06:30:00'))
(Timestamp('2018-05-17 07:30:00'), Timestamp('2018-05-17 16:30:00'))
(Timestamp('2018-05-18 05:00:00'), Timestamp('2018-05-18 06:30:00'))
(Timestamp('2018-05-18 08:30:00'), Timestamp('2018-05-18 09:00:00'))
(Timestamp('2018-05-18 16:30:00'),

(Timestamp('2018-06-26 05:00:00'), Timestamp('2018-06-26 17:00:00'))
(Timestamp('2018-06-26 22:30:00'), Timestamp('2018-06-26 23:30:00'))
(Timestamp('2018-06-27 06:00:00'), Timestamp('2018-06-27 07:00:00'))
(Timestamp('2018-06-27 12:00:00'), Timestamp('2018-06-27 17:00:00'))
(Timestamp('2018-06-27 19:00:00'), Timestamp('2018-06-27 20:00:00'))
(Timestamp('2018-06-28 01:30:00'), Timestamp('2018-06-28 02:00:00'))
(Timestamp('2018-06-28 13:00:00'), Timestamp('2018-06-28 13:30:00'))
(Timestamp('2018-06-28 15:30:00'), Timestamp('2018-06-28 16:30:00'))
(Timestamp('2018-06-29 06:00:00'), Timestamp('2018-06-29 07:30:00'))
(Timestamp('2018-06-29 14:30:00'), Timestamp('2018-06-29 17:30:00'))
(Timestamp('2018-06-29 19:30:00'), Timestamp('2018-06-29 20:00:00'))
(Timestamp('2018-06-30 05:30:00'), Timestamp('2018-06-30 08:00:00'))
(Timestamp('2018-06-30 15:30:00'), Timestamp('2018-06-30 20:00:00'))
(Timestamp('2018-06-30 21:30:00'), Timestamp('2018-06-30 23:00:00'))
(Timestamp('2018-07-01 00:00:00'),

...

(Timestamp('2021-01-31 14:30:00'), Timestamp('2021-02-01 05:00:00'))
(Timestamp('2021-02-01 18:30:00'), Timestamp('2021-02-01 19:30:00'))
(Timestamp('2021-02-01 21:00:00'), Timestamp('2021-02-01 23:00:00'))
(Timestamp('2021-02-02 05:00:00'), Timestamp('2021-02-02 05:30:00'))
(Timestamp('2021-02-02 15:30:00'), Timestamp('2021-02-02 19:00:00'))
(Timestamp('2021-02-02 23:00:00'), Timestamp('2021-02-02 23:30:00'))
(Timestamp('2021-02-03 10:30:00'), Timestamp('2021-02-03 11:00:00'))
(Timestamp('2021-02-03 12:00:00'), Timestamp('2021-02-03 18:00:00'))
(Timestamp('2021-02-04 04:00:00'), Timestamp('2021-02-04 04:30:00'))
(Timestamp('2021-02-04 05:30:00'), Timestamp('2021-02-04 06:00:00'))
(Timestamp('2021-02-04 10:30:00'), Timestamp('2021-02-04 14:00:00'))
(Timestamp('2021-02-04 15:00:00'), Timestamp('2021-02-04 16:30:00'))
(Timestamp('2021-02-05 05:30:00'), Timestamp('2021-02-05 06:00:00'))
(Timestamp('2021-02-05 07:00:00'), Timestamp('2021-02-05 10:30:00'))
(Timestamp('2021-02-05 18:30:00'),

(Timestamp('2021-03-27 19:30:00'), Timestamp('2021-03-27 22:00:00'))
(Timestamp('2021-03-28 05:00:00'), Timestamp('2021-03-28 06:00:00'))
(Timestamp('2021-03-28 12:00:00'), Timestamp('2021-03-28 12:30:00'))
(Timestamp('2021-03-28 15:30:00'), Timestamp('2021-03-28 17:00:00'))
(Timestamp('2021-03-29 03:00:00'), Timestamp('2021-03-29 05:00:00'))
(Timestamp('2021-03-29 16:00:00'), Timestamp('2021-03-29 16:30:00'))
(Timestamp('2021-03-30 04:30:00'), Timestamp('2021-03-30 05:30:00'))
(Timestamp('2021-03-30 15:30:00'), Timestamp('2021-03-30 17:00:00'))
(Timestamp('2021-03-31 04:30:00'), Timestamp('2021-03-31 05:30:00'))
(Timestamp('2021-03-31 11:30:00'), Timestamp('2021-03-31 12:30:00'))
(Timestamp('2021-03-31 14:00:00'), Timestamp('2021-03-31 16:30:00'))
(Timestamp('2021-04-01 04:30:00'), Timestamp('2021-04-01 06:00:00'))
(Timestamp('2021-04-01 13:00:00'), Timestamp('2021-04-01 17:00:00'))
(Timestamp('2021-04-02 13:00:00'), Timestamp('2021-04-02 16:30:00'))
(Timestamp('2021-04-03 13:30:00'),

(Timestamp('2021-05-24 04:30:00'), Timestamp('2021-05-24 06:30:00'))
(Timestamp('2021-05-24 14:00:00'), Timestamp('2021-05-24 16:00:00'))
(Timestamp('2021-05-25 05:00:00'), Timestamp('2021-05-25 07:00:00'))
(Timestamp('2021-05-25 08:00:00'), Timestamp('2021-05-25 09:00:00'))
(Timestamp('2021-05-25 11:00:00'), Timestamp('2021-05-25 11:30:00'))
(Timestamp('2021-05-26 04:30:00'), Timestamp('2021-05-26 07:00:00'))
(Timestamp('2021-05-26 13:00:00'), Timestamp('2021-05-26 16:30:00'))
(Timestamp('2021-05-27 03:30:00'), Timestamp('2021-05-27 04:00:00'))
(Timestamp('2021-05-27 05:00:00'), Timestamp('2021-05-27 06:30:00'))
(Timestamp('2021-05-27 13:30:00'), Timestamp('2021-05-27 14:30:00'))
(Timestamp('2021-05-27 15:30:00'), Timestamp('2021-05-27 17:00:00'))
(Timestamp('2021-05-28 05:00:00'), Timestamp('2021-05-28 06:30:00'))
(Timestamp('2021-05-28 08:30:00'), Timestamp('2021-05-28 09:30:00'))
(Timestamp('2021-05-28 12:30:00'), Timestamp('2021-05-28 13:30:00'))
(Timestamp('2021-05-28 14:30:00'),

(Timestamp('2021-07-12 05:30:00'), Timestamp('2021-07-12 08:30:00'))
(Timestamp('2021-07-12 15:30:00'), Timestamp('2021-07-12 16:30:00'))
(Timestamp('2021-07-12 20:30:00'), Timestamp('2021-07-12 21:00:00'))
(Timestamp('2021-07-13 06:00:00'), Timestamp('2021-07-13 07:30:00'))
(Timestamp('2021-07-13 13:30:00'), Timestamp('2021-07-13 14:30:00'))
(Timestamp('2021-07-13 16:00:00'), Timestamp('2021-07-13 17:00:00'))
(Timestamp('2021-07-14 05:30:00'), Timestamp('2021-07-14 10:00:00'))
(Timestamp('2021-07-14 16:30:00'), Timestamp('2021-07-14 17:00:00'))
(Timestamp('2021-07-15 06:00:00'), Timestamp('2021-07-15 07:00:00'))
(Timestamp('2021-07-15 15:30:00'), Timestamp('2021-07-15 17:30:00'))
(Timestamp('2021-07-16 05:30:00'), Timestamp('2021-07-16 08:30:00'))
(Timestamp('2021-07-16 16:00:00'), Timestamp('2021-07-16 16:30:00'))
(Timestamp('2021-07-17 06:00:00'), Timestamp('2021-07-17 08:30:00'))
(Timestamp('2021-07-17 15:30:00'), Timestamp('2021-07-17 16:00:00'))
(Timestamp('2021-07-17 21:00:00'),

,Time (UTC+10),Regions VIC Trading Price ($/MWh),Raw Power (MW),Market Dispatch (MWh),Opening Capacity (MWh),Closing Capacity (MWh),Status
0,2018-01-01 00:30:00,92.46,0.0,0.0,0.0,0.0,Nothing
1,2018-01-01 01:00:00,87.62,0.0,0.0,0.0,0.0,Nothing
2,2018-01-01 01:30:00,73.08,0.0,0.0,0.0,0.0,Nothing
3,2018-01-01 02:00:00,70.18,0.0,0.0,0.0,0.0,Nothing
4,2018-01-01 02:30:00,67.43,0.0,0.0,0.0,0.0,Nothing
...,...,...,...,...,...,...,...
63451,2021-08-14 22:00:00,49.93,-300.0,-150.0,0.0,135.0,Charge
63452,2021-08-14 22:30:00,62.86,270.0,121.5,135.0,0.0,Discharge
63453,2021-08-14 23:00:00,32.26,0.0,0.0,0.0,0.0,Nothing
63454,2021-08-14 23:30:00,25.10,0.0,0.0,0.0,0.0,Nothing


In [327]:
max_charge = []
min_charge = []
flags = []
for i in range(len(new_full)):
    max_c = -min(300, (580 - new_full.loc[i, 'Opening Capacity (MWh)'])/0.9 * 2)
    min_c = min(300, new_full.loc[i, 'Opening Capacity (MWh)'] * 2)
    max_charge.append(max_c)
    min_charge.append(min_c)
    
    #if float(new_full.loc[i, 'Raw Power (MW)']) < float(max_c[i]):
    #    flags.append(1)
    #elif new_full.loc[i, 'Raw Power (MW)'] > min_c[i]:
    #    flags.append(2)
    #else:
    #    flags.append(0)

new_full['Max Charge'] = pd.Series(max_charge)
new_full['Max Discharge'] = pd.Series(min_charge)

for i in range(len(new_full)):
    if new_full.loc[i, 'Raw Power (MW)'] < new_full.loc[i, 'Max Charge']:
        flags.append(1)
    elif new_full.loc[i, 'Raw Power (MW)'] > new_full.loc[i, 'Max Discharge']:
        flags.append(2)
    else:
        flags.append(0)
        
new_full['Flags'] = pd.Series(flags)
errors = new_full[new_full['Flags'] != 0]
errors

,Time (UTC+10),Regions VIC Trading Price ($/MWh),Raw Power (MW),Market Dispatch (MWh),Opening Capacity (MWh),Closing Capacity (MWh),Status,Max Charge,Max Discharge,Flags


In [293]:
errors.to_excel('../../preprocessed_data/First Algorithm/ERRORSlocalMaximisation.xlsx', index=False)

In [329]:
new_full.to_excel('../../preprocessed_data/First Algorithm/localMaximisation_without2ndOpti.xlsx', index=False)

In [328]:
new_full[new_full["Opening Capacity (MWh)"] != new_full["Closing Capacity (MWh)"].shift()]

,Time (UTC+10),Regions VIC Trading Price ($/MWh),Raw Power (MW),Market Dispatch (MWh),Opening Capacity (MWh),Closing Capacity (MWh),Status,Max Charge,Max Discharge,Flags
0,2018-01-01 00:30:00,92.46,0.0,0.0,0.0,0.0,Nothing,-300.0,0.0,0
